# Amazon Bedrock CRIS Manager Demo

This notebook demonstrates the functionality of the CRISManager class for managing Amazon Bedrock Cross-Region Inference (CRIS) model information.

## Overview

The CRISManager provides:
- Download of AWS Bedrock CRIS documentation
- Parsing of HTML expandable sections containing model information
- Extraction of inference profile IDs and region mappings
- JSON serialization with timestamps
- Query methods for source/destination regions
- Caching mechanisms for efficient data access

In [ ]:
# Import required modules
import sys
import logging
from pathlib import Path
from datetime import datetime
import json

# Add the src directory to the Python path
sys.path.append('../src')

from bedrock.CRISManager import CRISManager, CRISManagerError
from bedrock.models.cris_structures import CRISModelInfo, CRISCatalog, CRISInferenceProfile

# Configure logging to see what's happening
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("✅ CRISManager imported successfully!")

## 1. Basic Usage

Let's start with basic CRISManager usage - downloading and parsing CRIS documentation.

In [ ]:
# Create a CRISManager instance with default settings
cris_manager = CRISManager()

print(f"CRISManager configuration:")
print(f"  HTML output: {cris_manager.html_output_path}")
print(f"  JSON output: {cris_manager.json_output_path}")
print(f"  Documentation URL: {cris_manager.documentation_url}")
print(f"  Manager: {cris_manager}")

In [ ]:
# Download and parse CRIS documentation
# Note: This will download from AWS, so it requires internet connectivity
print("🔄 Downloading and parsing CRIS documentation...")

try:
    catalog = cris_manager.refresh_cris_data(force_download=True)
    print(f"✅ Successfully processed CRIS data!")
    print(f"   Found {catalog.model_count} CRIS models")
    print(f"   Retrieved at: {catalog.retrieval_timestamp}")
except CRISManagerError as e:
    print(f"❌ Error processing CRIS data: {e}")
    # For demo purposes, let's create some mock data
    print("📝 Creating mock data for demonstration...")
    
    # Create mock CRIS data with new structure
    from bedrock.models.cris_structures import CRISModelInfo, CRISCatalog, CRISInferenceProfile
    
    # Create inference profiles for Nova Lite model
    nova_lite_us_profile = CRISInferenceProfile(
        inference_profile_id="us.amazon.nova-lite-v1:0",
        region_mappings={
            "us-west-2": ["us-east-1", "us-east-2", "us-west-2"],
            "us-east-2": ["us-east-1", "us-east-2", "us-west-2"],
            "us-east-1": ["us-east-1", "us-east-2", "us-west-2"]
        }
    )
    
    nova_lite_eu_profile = CRISInferenceProfile(
        inference_profile_id="eu.amazon.nova-lite-v1:0",
        region_mappings={
            "eu-west-1": ["eu-central-1", "eu-west-1", "eu-west-3"],
            "eu-central-1": ["eu-central-1", "eu-west-1", "eu-west-3"]
        }
    )
    
    # Create inference profiles for Claude model
    claude_us_profile = CRISInferenceProfile(
        inference_profile_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
        region_mappings={
            "us-west-2": ["us-east-1", "us-west-2"],
            "us-east-1": ["us-east-1", "us-west-2"]
        }
    )
    
    claude_eu_profile = CRISInferenceProfile(
        inference_profile_id="eu.anthropic.claude-3-5-sonnet-20241022-v2:0",
        region_mappings={
            "eu-west-1": ["eu-central-1", "eu-west-1"]
        }
    )
    
    mock_models = {
        "Nova Lite": CRISModelInfo(
            model_name="Nova Lite",
            inference_profiles={
                "us.amazon.nova-lite-v1:0": nova_lite_us_profile,
                "eu.amazon.nova-lite-v1:0": nova_lite_eu_profile
            }
        ),
        "Claude 3.5 Sonnet": CRISModelInfo(
            model_name="Claude 3.5 Sonnet",
            inference_profiles={
                "us.anthropic.claude-3-5-sonnet-20241022-v2:0": claude_us_profile,
                "eu.anthropic.claude-3-5-sonnet-20241022-v2:0": claude_eu_profile
            }
        )
    }
    
    catalog = CRISCatalog(
        retrieval_timestamp=datetime.now(),
        cris_models=mock_models
    )
    
    # Set the catalog in the manager
    cris_manager._cached_catalog = catalog
    
    print(f"✅ Mock CRIS data created with {catalog.model_count} models")
    print(f"   Nova Lite has {len(catalog.cris_models['Nova Lite'].inference_profiles)} inference profiles")
    print(f"   Claude 3.5 Sonnet has {len(catalog.cris_models['Claude 3.5 Sonnet'].inference_profiles)} inference profiles")

## 2. Exploring CRIS Model Data

Let's explore the structure and content of the CRIS model data.

In [ ]:
# Get basic statistics about the CRIS catalog
print(f"📊 CRIS Catalog Statistics:")
print(f"   Total models: {cris_manager.get_model_count()}")
print(f"   Model names: {cris_manager.get_model_names()}")
print(f"   All source regions: {cris_manager.get_all_source_regions()}")
print(f"   All destination regions: {cris_manager.get_all_destination_regions()}")

In [ ]:
# Display detailed information for each model
print("🔍 Detailed Model Information:")
print("=" * 60)

for model_name in cris_manager.get_model_names():
    print(f"\n📋 Model: {model_name}")
    
    # Get model info from catalog
    model_info = catalog.cris_models[model_name]
    
    # Show primary inference profile (for backward compatibility)
    primary_profile_id = model_info.inference_profile_id
    print(f"   Primary Inference Profile ID: {primary_profile_id}")
    
    # Show all inference profiles
    print(f"   All Inference Profiles: {list(model_info.inference_profiles.keys())}")
    
    print(f"   Source Regions (all profiles): {model_info.get_source_regions()}")
    print(f"   Destination Regions (all profiles): {model_info.get_destination_regions()}")
    
    # Show detailed inference profile information
    print(f"   Inference Profile Details:")
    for profile_id, profile_info in model_info.inference_profiles.items():
        print(f"     🔹 {profile_id}:")
        print(f"        Source regions: {profile_info.get_source_regions()}")
        print(f"        Destination regions: {profile_info.get_destination_regions()}")
        for source, destinations in profile_info.region_mappings.items():
            print(f"          {source} → {destinations}")
    
    print("-" * 40)

## 3. Querying by Source Region

Find all models available from specific source regions.

In [ ]:
# Query models by source region
source_regions_to_test = ["us-west-2", "us-east-1", "eu-west-1"]

for source_region in source_regions_to_test:
    print(f"\n🌍 Models available from source region '{source_region}':")
    
    try:
        models = cris_manager.get_models_by_source_region(source_region=source_region)
        
        if models:
            for model_name, model_info in models.items():
                destinations = model_info.get_destinations_for_source(source_region=source_region)
                print(f"   ✅ {model_name}: {model_info.inference_profile_id}")
                print(f"      → Can route to: {destinations}")
                
                # Show which specific profiles support this source region
                supporting_profiles = model_info.get_profiles_for_source_region(source_region=source_region)
                print(f"      📋 Supporting profiles: {supporting_profiles}")
        else:
            print(f"   ❌ No models available from {source_region}")
            
    except CRISManagerError as e:
        print(f"   ❌ Error querying source region: {e}")

## 4. Querying by Destination Region

Find all models that can route requests to specific destination regions.

In [ ]:
# Query models by destination region
destination_regions_to_test = ["us-east-1", "us-west-2", "eu-central-1"]

for dest_region in destination_regions_to_test:
    print(f"\n🎯 Models that can route to destination region '{dest_region}':")
    
    try:
        models = cris_manager.get_models_by_destination_region(destination_region=dest_region)
        
        if models:
            for model_name, model_info in models.items():
                # Find which source regions can route to this destination
                source_regions = []
                for source, destinations in model_info.region_mappings.items():
                    if dest_region in destinations:
                        source_regions.append(source)
                
                print(f"   ✅ {model_name}: {model_info.inference_profile_id}")
                print(f"      ← Can be called from: {source_regions}")
                
                # Show which specific profiles can route to this destination
                supporting_profiles = model_info.get_profiles_for_destination_region(destination_region=dest_region)
                print(f"      📋 Supporting profiles: {supporting_profiles}")
        else:
            print(f"   ❌ No models can route to {dest_region}")
            
    except CRISManagerError as e:
        print(f"   ❌ Error querying destination region: {e}")

## 5. JSON Output Format

Examine the JSON output format and structure.

In [ ]:
# Display JSON structure
print("📄 CRIS JSON Output Format:")
print("=" * 40)

# Convert catalog to dictionary for display
catalog_dict = catalog.to_dict()

# Pretty print the JSON structure
print(json.dumps(catalog_dict, indent=2, default=str))

In [ ]:
# Show the field constants being used
from bedrock.models.cris_constants import CRISJSONFields

print("🏷️ JSON Field Constants (for maintainability):")
print(f"   Retrieval Timestamp: '{CRISJSONFields.RETRIEVAL_TIMESTAMP}'")
print(f"   CRIS Root: '{CRISJSONFields.CRIS}'")
print(f"   Model Name: '{CRISJSONFields.MODEL_NAME}'")
print(f"   Inference Profiles: '{CRISJSONFields.INFERENCE_PROFILES}'")
print(f"   Inference Profile ID: '{CRISJSONFields.INFERENCE_PROFILE_ID}' (backward compatibility)")
print(f"   Region Mappings: '{CRISJSONFields.REGION_MAPPINGS}'")

print("\n💡 All JSON field access uses these constants to ensure maintainability!")
print("\n🔧 The new structure supports multiple inference profiles per model:")
print("   - Each model can have US, EU, APAC variants")
print("   - Each inference profile has its own region mappings")
print("   - Backward compatibility is maintained with primary profile access")

## 6. Configuration and Customization

Show different configuration options.

In [ ]:
# Demonstrate different configuration options
print("⚙️ Configuration Options:")

# Custom paths
custom_manager = CRISManager(
    html_output_path=Path("../docs/custom_cris.htm"),
    json_output_path=Path("../docs/custom_cris.json"),
    download_timeout=60
)

print(f"   Custom HTML path: {custom_manager.html_output_path}")
print(f"   Custom JSON path: {custom_manager.json_output_path}")
print(f"   Documentation URL: {custom_manager.documentation_url}")

# Show default vs custom
default_manager = CRISManager()
print(f"\n🔄 Comparison with defaults:")
print(f"   Default HTML: {default_manager.html_output_path}")
print(f"   Custom HTML:  {custom_manager.html_output_path}")
print(f"   Default JSON: {default_manager.json_output_path}")
print(f"   Custom JSON:  {custom_manager.json_output_path}")

## 7. Summary and Best Practices

Summary of CRISManager capabilities and usage recommendations.

In [ ]:
print("📋 CRISManager Summary:")
print("=" * 50)

print("\n✅ Key Features Demonstrated:")
features = [
    "Download and parse AWS Bedrock CRIS documentation",
    "Extract inference profile IDs and region mappings", 
    "Support for multiple regional inference profiles per model",
    "Query models by source and destination regions",
    "Caching mechanisms for performance",
    "Comprehensive error handling",
    "JSON serialization with timestamps",
    "Type-safe data structures",
    "Configurable file paths and URLs",
    "Production-ready logging",
    "Backward compatibility with legacy structure"
]

for i, feature in enumerate(features, 1):
    print(f"   {i}. {feature}")

print("\n💡 Best Practices:")
practices = [
    "Use caching to avoid unnecessary downloads",
    "Handle CRISManagerError exceptions properly",
    "Configure appropriate download timeouts",
    "Use named parameters for better readability",
    "Leverage type hints for better IDE support",
    "Monitor logs for download and parsing status",
    "Consider regional inference profiles for optimal routing"
]

for i, practice in enumerate(practices, 1):
    print(f"   {i}. {practice}")

print("\n🎯 Use Cases:")
use_cases = [
    "Cross-region inference setup and configuration",
    "Model availability checking across regions",
    "Automated deployment scripts for multi-region setups",
    "Cost optimization by choosing optimal regions",
    "Disaster recovery and failover planning",
    "Regional compliance and data residency requirements"
]

for i, use_case in enumerate(use_cases, 1):
    print(f"   {i}. {use_case}")

print("\n🆕 New in this Version:")
new_features = [
    "Fixed regional overwrite bug - no more mixed region mappings!",
    "Support for multiple inference profiles per model",
    "Proper separation of US, EU, APAC variants",
    "Enhanced JSON structure with inference_profiles field",
    "Backward compatibility maintained for existing code"
]

for i, feature in enumerate(new_features, 1):
    print(f"   {i}. {feature}")

print("\n🎉 CRISManager Demo Complete!")
print("   Ready for production use with comprehensive CRIS model management.")
print("   🔧 The regional overwrite bug has been fixed - inference profiles are now properly separated!")